In [ ]:
import pandas as pd
import warnings
import sklearn
from sklearn.preprocessing import StandardScaler
import numpy as np
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

In [ ]:
new_data = pd.read_csv("data_unseen.csv")

In [ ]:
new_data.head()

,msno,city,bd,gender,registered_via,registration_init_time,is_churn,total_payment_channels,change_in_payment_methods,payment_plan_days_mean,change_in_plan,plan_list_price_mean,actual_amount_paid_mean,is_auto_renew_mean,is_autorenew_change_flag,transaction_date_min,transaction_date_max,total_transactions,membership_expire_date_max,is_cancel_mean,is_cancel_change_flag,discount_mean,is_discount_mean,is_discount_max,amt_per_day_mean,membership_duration_mean,more_than_30_sum,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs,login_freq,last_login,registration_duration
0,X1AmJaNJ1bpGEgxLveRwBhxGHytaIHHuNfAAPFKCFg4=,1,6.3,2,13,2016-12-27,0,3,1,30.0,1,100.000000,100.000000,1.0,1,2016-12-27,2017-02-27,3,2017-03-26,0.0,0,0.0,0.0,0,3.333333,29.0,0,1.791759,1.581984,0.934417,0.000000,3.230200,3.219675,9.070,4,2017-02-04,89
1,5obk4z2LF+rxlMWfs6GSj7p1HyFUYCvrsO6itixJIxk=,22,33.0,1,9,2015-02-05,0,6,2,30.0,1,149.833333,149.833333,0.0,0,2016-06-18,2017-02-17,6,2017-03-19,0.0,0,0.0,0.0,0,4.994444,30.5,2,0.000000,0.000000,0.000000,0.000000,5.296980,2.029523,5.953,12,2017-01-16,773
2,9eoVMtsXwWvmDtvBHzLTV5gegf59sdeuTvdbhnl77nk=,1,6.3,2,13,2016-12-27,0,3,1,30.0,1,129.000000,129.000000,1.0,1,2016-12-27,2017-02-27,3,2017-03-26,0.0,0,0.0,0.0,0,4.300000,29.0,0,1.838110,0.346574,0.000000,0.693147,1.758377,2.405943,7.695,4,2017-02-05,89
3,E7AzsekzVHL4d4ropO9g6HOwXUfsfrdM6FINWb6xTbs=,11,28.0,0,4,2017-01-09,0,2,1,30.0,1,180.000000,180.000000,1.0,1,2017-01-13,2017-02-13,2,2017-03-13,0.0,0,0.0,0.0,0,6.000000,29.5,1,1.069166,0.173287,0.000000,0.000000,1.543446,1.931663,7.145,4,2017-02-17,63
4,t2pAx7JJ+u9/tADRTYPb3EBqcDDk4M9JZIKG01FaagI=,5,28.0,0,4,2017-01-11,1,1,1,30.0,1,180.000000,180.000000,0.0,0,2017-01-15,2017-01-15,1,2017-02-14,0.0,0,0.0,0.0,0,6.000000,30.0,0,0.748933,0.346574,0.000000,0.173287,3.310596,3.338761,9.030,4,2017-01-22,34


In [ ]:
def get_data_prepared_for_modeling(dataframe,scale_method='standard',date_columns=None,corr_threshold=0.90):
    print(len(dataframe.columns))
    # removingmulti-colinearity 

    # Create correlation matrix
    corr_matrix = dataframe.corr().abs()
    # Select upper triangle of correlation matrix
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
    # Find features with correlation greater than 0.95
    to_drop = [column for column in upper.columns if any(upper[column] > corr_threshold)]
    print(to_drop)
    # Drop features 
    dataframe.drop(to_drop, axis=1, inplace=True)
    print(len(dataframe.columns))

    #date transformation 
    features = ["day","month","year","weekday"]
    date_data = dataframe[date_columns]
    for eachcol in date_data:
        date_data[eachcol] = date_data[eachcol].astype('str')
        date_data[eachcol] = pd.to_datetime(date_data[eachcol]) 
        #column_name
        for eachfeature in features:
            col_name = f"{eachcol}_{eachfeature}"
  
            if eachfeature == 'day':
                date_data[col_name] = date_data[eachcol].dt.day
                # result[col_name] = result[col_name].astype('int64')
            elif eachfeature == 'month':
                date_data[col_name] = date_data[eachcol].dt.month
                # result[col_name] = result[col_name].astype('int64')
            elif eachfeature == 'year':
                date_data[col_name] = date_data[eachcol].dt.year
                # result[col_name] = result[col_name].astype('int64')
            elif eachfeature == 'weekday':
                date_data[col_name] = date_data[eachcol].dt.weekday
    date_data.drop(date_columns,axis=1,inplace=True)
    date_data = date_data.where(date_data.isna(), date_data.astype(str))
    final_date = pd.get_dummies(date_data, drop_first=True,dtype='int16')
    # print(pd.get_dummies(date_data, drop_first=True,dtype='int16')) 
  
    #scaling
    column_to_scale = new_data.select_dtypes(include=['float64','int64']).columns.drop('is_churn')
    transformer = StandardScaler().fit(dataframe[column_to_scale])
    scaled_data = pd.DataFrame(transformer.transform(dataframe[column_to_scale]),columns=column_to_scale)

    #Combining 
    final_df = pd.concat([scaled_data,final_date,dataframe['is_churn']],axis=1)

    #Splitting X,y 
    X = final_df.drop('is_churn',axis=1)
    y = final_df[['is_churn']]
    return X,y

In [ ]:
X,y = get_data_prepared_for_modeling(new_data,date_columns = ['registration_init_time','transaction_date_min','transaction_date_max','membership_expire_date_max','last_login'])

37
['plan_list_price_mean', 'actual_amount_paid_mean', 'is_autorenew_change_flag', 'total_transactions', 'total_secs']
32


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import lightgbm as lgb

def get_train_model(X,y):
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)
  #Model Training
  clf = lgb.LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               device='gpu', importance_type='split', learning_rate=0.1,
               max_depth=-1, min_child_samples=20, min_child_weight=0.001,
               min_split_gain=0.0, n_estimators=100, n_jobs=-1, num_leaves=31,
               objective=None, random_state=42, reg_alpha=0.0, reg_lambda=0.0,
               silent='warn', subsample=1.0, subsample_for_bin=200000,
               subsample_freq=0)
  clf.fit(X_train, y_train)
  # predict the results
  y_pred=clf.predict(X_test)
  # view accuracy
  accuracy=accuracy_score(y_pred, y_test)
  print('LightGBM Model accuracy score: {0:0.4f}'.format(accuracy_score(y_test, y_pred)))
  print(classification_report(y_test, y_pred))
  print(confusion_matrix(y_test, y_pred))
  return clf

In [ ]:
def get_predict(model,x):
  return model.predict_proba(x)

In [ ]:
# Full Pipeline 

#new data coming in with Y values 
X,y = get_data_prepared_for_modeling(new_data,date_columns = ['registration_init_time','transaction_date_min','transaction_date_max','membership_expire_date_max','last_login'])

# Train the model with X & y 
model = get_train_model(X,y)

#Predict Object 
predictions = get_predict(model,X)

In [ ]:
import lightgbm as lgbm
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn.model_selection import StratifiedKFold

num_splits = 20
strat_kf = StratifiedKFold(n_splits=num_splits, shuffle=True, random_state=42)

scores = np.empty(N_SPLITS)
for idx, (train_idx, test_idx) in enumerate(strat_kf.split(X, y)):
    print("=" * 10 + f"Training fold {idx}" + 10 * "=")

    X_train, X_val = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[test_idx]
    eval_set = [(X_val, y_val)]

    lgbm_clf = lgbm.LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1,
               max_depth=-1, min_child_samples=20, min_child_weight=0.001,
               min_split_gain=0.0, n_estimators=100, n_jobs=-1, num_leaves=31,
               objective=None, random_state=42, reg_alpha=0.0, reg_lambda=0.0,
               silent='warn', subsample=1.0, subsample_for_bin=200000,
               subsample_freq=0)
    lgbm_clf.fit(X_train, y_train)
    # predict the results
    y_pred=lgbm_clf.predict(X_val)
    # view accuracy
    accuracy=accuracy_score(y_val, y_pred)
    print('LightGBM Model accuracy score: {0:0.4f}'.format(accuracy))
    print(classification_report(y_val, y_pred))
    print(confusion_matrix(y_val, y_pred))

==========Training fold 0==========
LightGBM Model accuracy score: 0.9654
              precision    recall  f1-score   support

           0       0.98      0.99      0.98       755
           1       0.78      0.69      0.73        55

    accuracy                           0.97       810
   macro avg       0.88      0.84      0.86       810
weighted avg       0.96      0.97      0.96       810

[[744  11]
 [ 17  38]]
==========Training fold 1==========
LightGBM Model accuracy score: 0.9580
              precision    recall  f1-score   support

           0       0.98      0.98      0.98       755
           1       0.70      0.67      0.69        55

    accuracy                           0.96       810
   macro avg       0.84      0.83      0.83       810
weighted avg       0.96      0.96      0.96       810

[[739  16]
 [ 18  37]]
==========Training fold 2==========
LightGBM Model accuracy score: 0.9580
              precision    recall  f1-score   support

           0       0.98